# Домашнее задание №1 - EDA. Уменьшение размерности

В этой домашке вам предстоит немножко углубиться в изучение **показателей крови**, а именно посмотреть по каким параметрам общего анализа крови ставится тот или иной **вид анемии**. 

Для анализа мы подобрали большой датасет, но дадим поменьше. Вам нужно будет поработать с **EDA, статистикой и уменьшением размерности**. Мы всё покажем, а вы всё сделаете - не волнуйтесь!

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from scipy import stats

Мы будем использовать общедоступный датасет **Anemia Disease Dataset**. 
Любую работу в анализе данных стоит начинать с самостоятельного изучения этих самых данных, с которыми вам нужно поработать. Почитайте описание этого датасета по ссылкам ниже (оно вам точно пригодится), поймите в каких колонках что представлено, какого характера данные в них должны быть (минимальное/максимальное значение, число или текст), сколько тут пациентов и есть ли какие-то тонкости относительно качества данных. Что есть признаки, а что есть таргет для будущих анализа и предсказаний.

[Датасет на kaggle](https://www.kaggle.com/code/serhathoca/anemia-disease-dataset/notebook)

[Публикация, из которой взят датасет](https://www.sciencedirect.com/science/article/abs/pii/S174680942030361X)

Но помните: *Ушам своим не верьте вовсе, а глазам — только наполовину.* © «Убийство на улице Морг» Эдгар Аллан По. 

Так что даже если у вас есть супер-описание вашего нового датасета, то на вашей совести проверить эти данные перед работой (даже если его проверяли специалисты и эксперты). Делайте EDA, чтобы узнать что-то новое о данных!

**Небольшое отступление**: старайтесь приводить наглядные и понятные графики, с которых легко и быстро можно считать информацию о ваших результатах. Помните, мы говорили про ширину бинов и масштаб гистограмм на лекции? График должен содержать подписи осей, название графика, шкалы, цвета, легенды и т.д. Если у вас много однотипных графиков, лучше оформлять их в виде панели, т.е. с использованием `fig, ax = plt.subplots()`.

Best practice: *Лучший график - это тот, который можно разобрать, не читая текст.*

> Так, я как учитель напишу что-то тут

## Часть 0. Загружаем данные (20 баллов)

Загружаем датасет

In [36]:
df = pd.read_csv('anemia_dataset.csv', index_col = 0)
df

,GENDER,WBC,NE#,LY#,MO#,EO#,BA#,RBC,HGB,HCT,...,SDTSD,TSD,FERRITTE,FOLATE,B12,All_Class,HGB_Anemia_Class,Iron_anemia_Class,Folate_anemia_class,B12_Anemia_class
0,1,10.630000,6.31,2.79,0.91,0.56,0.06,4.31,12.70,37.60,...,248.90,40.176778,194.000000,5.06,178.2,4,0,0,0,1
1,1,5.080000,2.50,1.87,0.43,0.26,0.02,4.34,12.80,36.90,...,348.48,33.482553,57.370000,9.88,197.7,4,0,0,0,1
2,1,13.680000,9.40,2.69,1.55,0.03,0.01,3.18,9.40,27.50,...,357.27,20.144429,114.200000,8.37,143.0,4,0,0,0,1
3,1,5.600000,3.94,0.83,0.54,0.26,0.03,3.35,10.50,31.40,...,360.60,27.731559,214.200000,6.39,139.9,4,0,0,0,1
4,1,3.570000,2.03,1.25,0.10,0.18,0.01,1.31,5.10,14.30,...,223.28,78.860623,303.400000,4.30,50.0,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,1,9.550000,5.88,3.04,0.55,0.03,0.05,4.81,11.86,38.03,...,622.00,5.948553,14.970000,16.01,1537.0,2,0,1,0,0
1421,0,8.436995,0.77,0.86,0.17,0.03,0.02,3.72,8.70,28.10,...,312.40,8.322663,22.995831,5.05,87.7,2,0,1,0,0
1422,0,7.180000,4.36,2.04,0.53,0.20,0.05,4.42,10.00,32.30,...,403.80,14.611194,8.030000,13.51,532.9,2,0,1,0,0
1423,0,4.020000,2.69,1.11,0.17,0.03,0.02,4.76,11.14,34.62,...,311.00,10.610932,221.800000,9.25,487.6,2,0,1,0,0


*(5 баллов)* Напишите **сколько фичей** и **сколько образцов** в данном датасете. Укажите **сколько классов** анемии здесь представлено. 

In [9]:
#your code is here

*(5 баллов)* Как вы поняли, в датасете есть колонки, отвечающие за тип анемии (*anemia_class*) - приведите их к виду **"одна колонка с категорией пациента в виде строки"**. Использовать колонку *All_Class* запрещено 

> **Доп балл (5 баллов)**: будет дан за использование методов melt/pivot*)

In [1]:
#your code is here

#Как учитель: Ты вообще в курсе что надо было кодить, не?

*(10 баллов)* Можете **добавить любой другой шаг** к анализу, который считаете нужным тут

In [40]:
#your code is here, if you want

Не забудем **удалить** уже ненужные колонки

In [16]:
#your code is here

## Часть 1. Анализ числовых признаков (30 баллов)

*(5 баллов)* Какие **числовые признаки** есть в этом датасете? И что они **означают**?

In [31]:
#your code is here

In [2]:
Пу-пу-пу как учитель

SyntaxError: invalid syntax (<ipython-input-2-c19135e0a158>, line 1)

*(20 баллов)* Проверьте какие переменные распределены **нормально**? **Лог-нормально**? Имеют **мультимодальное** распределение? Приведите соответствующие **графики** и в случае **нормальности** проведите соответствующий **стат.тест**. Предложите **объяснение** с биологической точки зрения, почему у некоторых переменных есть **несколько пиков**.

*Возможно, вам будет интересен модуль `scipy.stats`*

> **Доп балл *(5 баллов)***: за проведение стат.тестов для лог-нормального распределения.

> **Доп балл *(10 баллов)***: за проведение для любого распределения на ваш выбор по методу оценки параметров и генерации выборок

In [19]:
#your code is here

In [ ]:
#это поможет вам провести тест на нормальность

x = #your code is here
shapiro_test = stats.shapiro(x)

*(5 баллов)* Если вы хорошо построите **график распределения** переменной `B12`, то увидите заметный **пик** в области больших значений. **Подумайте** откуда мог взяться этот пик.

In [32]:
#your code is here

## Часть 2. Анализ категориальных признаков (20 баллов)

*(5 баллов)* Какие **категориальные признаки** есть в этом датасете? Они **порядковые** или **номинальные**?

In [30]:
#your code is here

*(5 баллов)* Придумайте и добавьте в ваш датасет **два дополнительных порядковых** признака.

In [33]:
#your code is here

*(5 баллов)* Убедитесь в **типах данных** каждого признака. Особенно обратите внимание на **порядковые**. Обычно это должны быть `pd.Categorical` с учётом упорядоченности.

In [ ]:
#your code is here

*(5 баллов)* Изобразите наглядные **боксплоты** для `B12` по одной из **категориальных** переменных. **Прокомментируйте** ваш результат

In [35]:
sns.boxplot(data=df, x='B12', y='YOUR VARIABLE')

#your code is here

## Часть 3. Группируем данные и анализируем (20 баллов)

*(10 баллов)* В датасете есть категориальная переменная `sex`. Определите все переменные, значение которых **зависит от пола**.

*ПыСы. Аккуратно формируйте нулевую и альтернативную гипотезу, не забудьте что вы проводите много тестов*

In [35]:
#your code is here

*(5 баллов)* В случае значимого отличия между группами **визуализируйте эти отличия** любым удобным и наглядным способом. Предложите **биологическую интерпретацию** наблюдаемых различий

In [35]:
#your code is here

Готовы к задачке посложнее? 

*(5 баллов)* Для значимых отличий визуализируйте их, **разбивая еще и по диагнозу**. Посмотрите, есть ли **отличия** между здоровыми и каким-либо классом пациентов с анемией.

In [41]:
#your code is here

## Часть 4. Поиск взаимосвязей в данных: Корреляции и Хитмапы (20 баллов)

*(5 баллов)* Для здоровых пациентов вычислите **корреляцию между числовыми признаками**. Какие пары признаков имеют **наибольший** коэффициент корреляции, а какие **наименьший**? Попробуйте **порассуждать** почему именно так.

In [42]:
#your code is here

*(10 баллов)* Постройте **heatmap**, в котором все попарно **не стат значимые** корреляции будут отмечены **серым**, а **стат значимые** цветом **в зависимости** от значения коэффициента корреляции.

In [ ]:
#your code is here

*(5 баллов)* В датасете есть **пара переменных с интересной зависимостью**. Найдите эту пару переменных, наглядно изобразите график, сформируйте **биологическую гипотезу** о природе зависимости. Какой коэффициент корреляции между этими переменными?

In [ ]:
#your code is here

## Часть 5. Уменьшение размерности: PCA (40 баллов)

Ура! Вы дошли до супер-интересной части!

*(10 баллов)*

Постройте **PCA** по числовым переменным для четырех версий датасета:
* в исходном виде
* с применением min-max стандартизации
* с применением z-стандартизации
* с применением robust стандартизации

И не забудьте **визуализировать** результаты.

In [ ]:
#создадим объект pca
pca = PCA()

#запускаем PCA на нашем датасете
principal_components = pca.fit_transform(df)

In [ ]:
#визуализируем результат PCA с помощью seaborn или matplotlib 
plt.scatter(principal_components[:, 1], principal_components[:, 2], alpha=0.5)

#теперь можете кастомизировать график на свой вкус

#your code is here

In [ ]:
#и попробуйте отшкалировать ваши данные и повторить анализ

#your code is here

*(5 баллов)* Посмотрите внимательно на изображения вариантов PCA. **Чем** они отличаются?

In [ ]:
#your code is here

*(10 баллов)* Выберите вариант, который на ваш взгляд, **лучший**. Объясните свой **выбор** и приведите **долю объясненной дисперсии** для первых 5 компонент. И постройте подобный **график** для ваших данных.

![variance_pca](variance_pca.jpg)

In [ ]:
#Получаем долю объяснённой дисперсии
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_cum = np.cumsum(explained_variance_ratio)

#your code is here

*(5 баллов)* Какие переменные дают **больший вклад** в первую компоненту? А во вторую?

In [ ]:
first_component = pca.components_[0]

#your code is here

*(5 баллов)* **Разукрасьте** ваш лучший вариант по:
* полу
* диагнозу
* статусу больной/здоровый

In [ ]:
#your code is here

*(5 баллов)* **Прокомментируйте** то что получилось

In [ ]:
#your code is here

**Отлично! Вы дошли до конца первой домашки!**

Поделитесь с нами вашими ощущениями об этой домашке, если хотите. Было это сложно или легко, понятно или запутанно, долго или быстро. Напишите что нового вы узнали. 

In [ ]:
#that's your turn

## Давайте знакомиться

Расскажите нам о вашем **любимом супергерое**. Marvel, DC, Archie Comics, Bubble Comics...что угодно

In [2]:
# I'm 🦇 ᗷᗩTᗰᗩᑎ 🦇 
Мне тож бэтмен нрав кнш

Ну такая себе домашка конечно, Оль, ты там получше в том году делала конечно. Совсем расслабилась, так нельзя. Соберись!